In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets,transforms
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# 定义超参数
input_size = 28  #图像的总尺寸28*28
num_classes = 10  #标签的种类数
num_epochs = 3  #训练的总循环周期
batch_size = 64  #一个撮（批次）的大小，64张图片

# 训练集
train_dataset = datasets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

# 测试集
test_dataset = datasets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

# 构建batch数据
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)  # shuffle=True设置打乱数据集
test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                           batch_size=batch_size,
                                           shuffle=True)

In [3]:
train_loader.dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [4]:
test_loader.dataset

Dataset MNIST
    Number of datapoints: 10000
    Root location: ./data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [5]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # 输入大小 (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # 灰度图
                out_channels=16,            # 要得到几多少个特征图
                kernel_size=5,              # 卷积核大小
                stride=1,                   # 步长
                padding=2,                  # 如果希望卷积后大小跟原来一样，需要设置padding=(kernel_size-1)/2 if stride=1
            ),                              # 输出的特征图为 (16, 28, 28)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(kernel_size=2),    # 进行池化操作（2x2 区域）, 输出结果为： (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # 下一个套餐的输入 (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # 输出 (32, 14, 14)
            nn.ReLU(),                      # relu层
            nn.MaxPool2d(2),                # 输出 (32, 7, 7)
        )
        self.out = nn.Linear(32 * 7 * 7, 10)   # 全连接层得到的结果

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)           # flatten操作，结果为：(batch_size, 32 * 7 * 7)
        output_ = self.out(x)
        return output_

In [6]:
sign = 1
def accuracy(predictions, labels):
    global sign
    if sign:
        print(torch.max(predictions.data, 1))  # [0]是values, [1]是indices
        sign = 0
    pred = torch.max(predictions.data, 1)[1]
    rights = pred.eq(labels.data.view_as(pred)).sum()
    return rights, len(labels)

In [9]:
# 实例化
net = CNN()
#损失函数
criterion = nn.CrossEntropyLoss()
#优化器
optimizer = optim.Adam(net.parameters(), lr=0.001) #定义优化器

sign_1 = 1
#开始训练循环
for epoch in range(num_epochs):
    #当前epoch的结果保存下来
    train_rights = []

    for batch_idx, (data, target) in enumerate(train_loader):  #针对容器中的每一个批进行循环, enumerate为枚举对象增加索引
        net.train()  # 设置训练模式
        output = net(data)
        global sign_1
        if sign_1:
            print(output)
            sign_1 = 0
        loss = criterion(output, target)
        optimizer.zero_grad()  # 清空过往梯度
        loss.backward()  # 反向传播, 计算当前梯度
        optimizer.step()  # 根据梯度更新网络参数
        right = accuracy(output, target)
        train_rights.append(right)


        if batch_idx % 100 == 0:

            net.eval()  # 设置评估模式
            val_rights = []

            for (data, target) in test_loader:
                output = net(data)
                right = accuracy(output, target)
                val_rights.append(right)

            #准确率计算
            train_r = (sum([tup[0] for tup in train_rights]), sum([tup[1] for tup in train_rights]))
            val_r = (sum([tup[0] for tup in val_rights]), sum([tup[1] for tup in val_rights]))

            print('当前epoch: {} [{}/{} ({:.0f}%)]\t损失: {:.6f}\t训练集准确率: {:.2f}%\t测试集正确率: {:.2f}%'.format(
                epoch,
                batch_idx * batch_size,
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.data,
                100. * train_r[0] / train_r[1],
                100. * val_r[0] / val_r[1]
                )
            )

tensor([[-3.6011e-03,  1.1758e-02, -2.3896e-02,  1.4614e-01,  6.5817e-02,
          8.7678e-02, -2.1038e-02, -2.6237e-02, -3.9168e-02,  6.4779e-03],
        [ 1.3759e-02,  2.1503e-02,  3.1078e-02,  1.1649e-01,  6.6238e-02,
          1.1441e-01, -5.4587e-02, -1.1195e-03, -3.2415e-02, -2.5177e-02],
        [-1.3687e-02,  7.2141e-02,  2.7150e-02,  1.4102e-01,  3.7290e-02,
          1.1292e-01, -7.8344e-02, -3.4294e-02, -1.1538e-01, -8.5344e-03],
        [-1.1710e-02,  7.4030e-02,  1.8525e-02,  1.5381e-01,  1.3818e-01,
          1.8693e-01, -3.2590e-02, -4.3030e-02, -1.0288e-01, -1.7793e-02],
        [ 3.7349e-02,  4.2081e-02,  1.9253e-03,  1.1443e-01,  8.7311e-02,
          1.1226e-01, -4.1393e-02, -7.0211e-02, -5.5416e-02, -1.8686e-02],
        [ 5.8491e-02,  6.0286e-02,  1.7469e-02,  1.3493e-01,  4.0788e-02,
          1.0006e-01, -5.7931e-02, -1.2654e-02, -5.7279e-02, -9.6553e-04],
        [-6.2352e-03,  5.4563e-02,  6.2756e-02,  1.5312e-01,  6.4357e-02,
          1.3231e-01, -2.7155e-0

In [11]:
x = torch.tensor([1., 2.], requires_grad=True)
# x: tensor([1., 2.], requires_grad=True)
y = 100*x
# y: tensor([100., 200.], grad_fn=<MulBackward0>)

loss = y.sum()  # tensor(300., grad_fn=<SumBackward0>)

# Compute gradients of the parameters respect to the loss
print(x.grad)     # None, 反向传播前，梯度不存在
loss.backward()
print(x.grad)     # tensor([100., 100.]) loss对y的梯度为1， 对x的梯度为100

optim = torch.optim.SGD([x], lr=0.001) # 随机梯度下降， 学习率0.001
print(x)        # tensor([1., 2.], requires_grad=True)
optim.step()  # 更新x
print(x)        # tensor([0.9000, 1.9000], requires_grad=True) 变化量=梯度X学习率 0.1=100*0.001

None
tensor([100., 100.])
tensor([1., 2.], requires_grad=True)
tensor([0.9000, 1.9000], requires_grad=True)
